In [2]:
import torch as th 
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from model import Generator, Discriminator, gradient_penalty
from utils import PolyGraphDataset, Transformed_PolyGraphDataset, CATEGORY_DICT

import numpy as np
import time 

In [3]:
""" 
Hyperparameters
"""

MAX_POLYGONS = 30

# Optimizer params
g_lr = 0.001 
d_lr = 0.001
b1 = 0.5 
b2 = 0.999  

# WGAN params
N_critic = 5            # nr of times to train discriminator more
lambda_gp = 10          # gradient penalty hyperpraram

# Training params
MAX_EPOCHS = 50
BATCH_SIZE = 32

# Network parameters
NOISE_SIZE = 128
HIDDEN_GENERATOR = [64, 32, 32]
OUTPUT_GENERATOR = MAX_POLYGONS * 2             # we want to output at most this many polygons per node (note [x1...y1...] format)

HIDDEN_DISCRIMINATOR = [64, 32, 16]

In [4]:
""" 
Model Definitions
"""
generator = Generator(input_dim=NOISE_SIZE + len(CATEGORY_DICT), 
                      output_dim=OUTPUT_GENERATOR, 
                      hidden_dims=HIDDEN_GENERATOR)

discriminator = Discriminator(input_dim=OUTPUT_GENERATOR, 
                              hidden_dims=HIDDEN_DISCRIMINATOR)

print(generator.module_list)
print(discriminator.module_list)

ModuleList(
  (0): TAGConv(141, 64, K=3)
  (1): TAGConv(64, 32, K=3)
  (2): TAGConv(32, 60, K=3)
)
ModuleList(
  (0): TAGConv(60, 64, K=3)
  (1): TAGConv(64, 32, K=3)
  (2): TAGConv(32, 16, K=3)
  (3): Linear(in_features=16, out_features=1, bias=True)
)


In [5]:
# Optimizers
optimizer_G = th.optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2)) 
optimizer_D = th.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))

In [6]:
def run_epoch(generator, discriminator, optimizer_g, optimizer_d, data_loader):

    losses_g, losses_d = [], []
    num_steps = 0

    # real = batch of Data() ex. [Data(), Data(), ...] is 1 batch
    for i, real in enumerate(data_loader):
        num_steps += 1
        start_time = time.time()

        # Input noise_data into generator            
        noise = th.randn((len(real.category), NOISE_SIZE))
        fake = generator(real, noise)     

        # fake.shape = (batch_size * nodes, output_features = 60)
        # We must turn this into appropriate (batch) input for the discriminator
        fake = Batch(geometry=fake, edge_index=real.edge_index, batch=real.batch)

        discriminator_fake = discriminator(fake)    # discriminator scores for fakes
        discriminator_real = discriminator(real)    # discriminator scores for reals
        
        gp = gradient_penalty(discriminator, real, fake)

        # Discriminator loss and train
        loss_discriminator = -(th.mean(discriminator_real) - th.mean(discriminator_fake)) + lambda_gp * gp
        
        discriminator.zero_grad() 
        loss_discriminator.backward() 
        optimizer_d.step()

        losses_d.append(loss_discriminator)

        # Only train Generator every 5 steps
        if num_steps % N_critic == 0:
            noise_g = th.randn((len(real.category), NOISE_SIZE))
            fake_g = generator(real, noise_g) 
            fake_g = Batch(geometry=fake_g, edge_index=real.edge_index, batch=real.batch)

            output = discriminator(fake_g).reshape(-1)      # discriminator scores for fake
            loss_generator = -th.mean(output)               # loss for genereator = the discriminators' judgement
                                                            # higher score = better
            generator.zero_grad()
            loss_generator.backward()
            optimizer_g.step()

            losses_g.append(loss_generator)

        if i % BATCH_SIZE == 0:
            batch_time = time.time() - start_time
            print("Batch {} took {:.2f} seconds.".format(i, batch_time))
    
    return sum(losses_g) / len(losses_g), sum(losses_d) / len(losses_d)

In [7]:
# Main training loop
def train(generator, discriminator, optimizer_g, optimizer_d, data_loader, max_epochs):
    start_time = time.time() 
    losses_g, losses_d = [], []

    for epoch in range(1, max_epochs + 1):
        epoch_time = time.time() 

        loss_g, loss_d = run_epoch(generator, discriminator, optimizer_g, optimizer_d, data_loader)
        losses_g.append(loss_g)
        losses_d.append(loss_d)

        # TODO: Evaluation and logging code??
        epoch_time = time.time() - epoch_time 
        total_time = time.time() - start_time
        print("Total runtime: {:.2f}, Epoch: {}, took: {:.2f} seconds, loss_g: {:.2f}, loss_d: {:.2f}".format(total_time, epoch, epoch_time, loss_g, loss_d))

        # save model every 10 epochs
        if epoch % 10:
            generator_path = r'C:\School\DELFT\Graph_ML_project\saved_models\generator.pt'
            discriminator_path = r'C:\School\DELFT\Graph_ML_project\saved_models\discriminator.pt'
            th.save(generator.state_dict(), generator_path)
            th.save(discriminator.state_dict(), discriminator_path)

    return losses_g, losses_d

In [8]:
path = r'C:\School\DELFT\Graph_ML_project\data\swiss-dwellings-v3.0.0'
dataset = Transformed_PolyGraphDataset(path)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
n_batches = len(dataloader)
print(n_batches)

losses_g, losses_d = train(generator, discriminator, optimizer_G, optimizer_D, dataloader, max_epochs=10)

294
Total runtime: 118.78, Epoch: 1, took: 118.78 seconds, loss_g: -413.52, loss_d: -4225.43


Testing model for batches of input and gradient penalty for batches

In [ ]:
from model import gradient_penalty

print(dataset[0])
test_list = [dataset[0] for _ in range(32)]
test_batch = Batch.from_data_list(test_list)
print(test_batch)

real = test_batch

noise = th.randn((len(real.category), NOISE_SIZE))
print("noise: ", noise.shape)
fake = generator(real, noise)     

# fake.shape = (batch_size * nodes, output_features = 60)
# We must turn this into appropriate (batch) input for the discriminator
fake = Batch(geometry=fake, edge_index=real.edge_index, batch=real.batch)

print("fake: ", fake.geometry.shape)
print("real: ", real.geometry.shape)

discriminator_fake = discriminator(fake)    # discriminator scores for fakes
discriminator_real = discriminator(real)    # discriminator scores for reals

print("fake: ", fake)
print("real: ", real)
print("discriminator score fake: ", discriminator_fake.shape)
print("discriminator score real: ", discriminator_real.shape)

gp = gradient_penalty(discriminator, real, fake)
print(gp)

Data(edge_index=[2, 72], geometry=[42, 60], category=[42, 13], num_nodes=42)


DataBatch(edge_index=[2, 2304], geometry=[1344, 60], category=[1344, 13], num_nodes=1344, batch=[1344], ptr=[33])
noise:  torch.Size([1344, 128])
fake:  torch.Size([1344, 60])
real:  torch.Size([1344, 60])
fake:  DataBatch(edge_index=[2, 2304], geometry=[1344, 60], batch=[1344])
real:  DataBatch(edge_index=[2, 2304], geometry=[1344, 60], category=[1344, 13], num_nodes=1344, batch=[1344], ptr=[33])
discriminator score fake:  torch.Size([32, 1])
discriminator score real:  torch.Size([32, 1])
tensor(0.5717, grad_fn=<MeanBackward0>)


Testing dataloader iteration

In [ ]:
from tqdm.auto import tqdm

dataset = Transformed_PolyGraphDataset(path)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
dataiter = iter(dataloader)

for _ in tqdm(range(n_batches)):
    batch = next(dataiter)
    print(batch)